## Segmenting and Clustering Neighbourhoods in Toronto

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
import folium  #map rendering library
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('All required libraries load complete')

All required libraries load complete


## PART 1: 'Scrape the Wikipedia page and Create a DataFrame'

### Extracting table from the weblink using Beautiful Soup and panda

In [20]:
#this script shows the output using the link below on 24th april 2020.

file_link = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
res = requests.get(file_link)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

### Saving the table as dataframe

In [21]:
new_df = pd.DataFrame(df[0])

### Checking number of cells with 'not assigned' in 'Borough' & 'Neighbourhood'

In [22]:
#checking how many cells have 'not assigned' in Borough 
print('No. of cells with \'not assigned\' in Borough: ',new_df[new_df.Borough == 'Not assigned'].Borough.value_counts()[0])

No. of cells with 'not assigned' in Borough:  77


In [23]:
new_df.shape

(288, 3)

In [24]:
#checking if there are null values in the columns
new_df.isnull().sum()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

In [25]:
print('Total Boroughs with not assigned : ',new_df[new_df.Borough == 'Not assigned'].shape[0])
print('Total Neighborhood with Not assigned values : ',new_df[new_df.Neighbourhood == 'Not assigned'].shape[0])

Total Boroughs with not assigned :  77
Total Neighborhood with Not assigned values :  78


### Getting the dataframe where Borough has no 'Not Assigned'

In [26]:
new_df = new_df[new_df.Borough != 'Not assigned']

In [27]:
print('Neighborhood with No assigned Borough: ',new_df[new_df.Neighbourhood == 'Not assigned'].shape[0])
print('Hence, Need to fix this neighbourhood as Borough')

Neighborhood with No assigned Borough:  1
Hence, Need to fix this neighbourhood as Borough


In [28]:
not_assigned = new_df[new_df.Neighbourhood == 'Not assigned']

not_assigned

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [29]:
#fixing the Not assigned in Neighbour with the Borough name

new_df.loc[not_assigned.index,'Neighbourhood'] = not_assigned.Borough[not_assigned.index[0]]

In [30]:
#checking the changes  
print('Current No assigned Neighbourhood in the data: ',new_df[new_df.Neighbourhood == 'Not assigned'].shape[0])

Current No assigned Neighbourhood in the data:  0


In [31]:
new_df = new_df.dropna().groupby(['Postcode','Borough']).agg(lambda x: ','.join(x)).reset_index()


In [32]:
new_df.loc[:,'Neighbourhood'] = new_df.Neighbourhood.apply(lambda x: x.replace(' / ',', '))


In [33]:
print('The number of rows of the dataframe: ', new_df.shape[0])

The number of rows of the dataframe:  103


# PART 2: 'Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name'

## I used OPTION 2 the provided Geospatail csv file becaue the Google Maps Geocoding API was totally unreliable while running for me
### Using Geospatial_data csv file to get the lat and long

In [35]:
Geospatial_data_filename = 'http://cocl.us/Geospatial_data'

Geospatial_Coord = pd.read_csv(Geospatial_data_filename)

Geospatial_Coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
Geospatial_Coord.shape

(103, 3)

### Merge two dataframes using 'Postal code' as a join

In [37]:
new_df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [38]:
Geospatial_Coord.dtypes


Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [39]:
# notice the postcode is different in new_df and Geospatial_Coord

new_df = new_df.rename(columns = {'Postcode':'Postal Code'})

### observation : Postal code and Postal Code different

In [40]:
# Joining the two dataframes to get the lat and long
lat_long = new_df.merge(Geospatial_Coord, how = 'inner', on=['Postal Code'])

In [41]:
lat_long


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [42]:
lat_long.shape


(103, 5)

## PART 3: Explore and cluster the neighborhoods in Toronto. Deciding to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

### Getting the dataframe where Borough has 'Toronto' in it

In [43]:
df_Toronto = lat_long[lat_long['Borough'].str.contains('Toronto')].reset_index(drop = True)


In [44]:
df_Toronto.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [45]:
df_Toronto.shape


(38, 5)

### Replicating the same analysis we did with New York City data

In [46]:
df_Toronto.head(5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Let's get the geographical location of Toronto

In [48]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application1")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude));

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with Neighbourhoods superimposed on top

In [49]:
neighbourhoods = df_Toronto[['Borough', 'Neighbourhood', 'Latitude','Longitude']]
neighbourhoods.head(5)

,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [50]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighbourhoods.


In [51]:
# create map of Toronto  using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto.save('map_toronto.html')
map_toronto 

### Define Foursquare Credentials and Version

In [55]:
CLIENT_ID = 'BCW0BQ1ILZUXBMX3FF52AYLGNIM0HV0BE0YMJIQVAOQ5GA0N' 
CLIENT_SECRET = 'TPA1LPD2KPRNAU1W3T2X1NWJXL0XAW1QZKR0NCNDAEF5WVZG'
VERSION = '20180605' # Foursquare API version

### Let's explore the first Neighbourhood in the dataframe

In [56]:
#Get the Neighbourhood name
df_Toronto.loc[0,'Neighbourhood']

'The Beaches'

In [57]:
# Get the neighbourhood's latitude and longitude values.

neighborhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 metres

In [58]:
#create the GET request URL
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=BCW0BQ1ILZUXBMX3FF52AYLGNIM0HV0BE0YMJIQVAOQ5GA0N&client_secret=TPA1LPD2KPRNAU1W3T2X1NWJXL0XAW1QZKR0NCNDAEF5WVZG&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [59]:
results = requests.get(url).json()


In [60]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Next to clean the json and structure it into pandas dataframe

In [61]:
json_normalize(results['response']['groups'][0]['items']).columns


C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.postalCode', 'venue.venuePage.id'],
      dtype='object')

In [62]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
print('flattened json column names: ',nearby_venues.columns)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


flattened json column names:  Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.postalCode', 'venue.venuePage.id'],
      dtype='object')


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


4 venues were returned by Foursquare.


### Explore neighbourhoods in Toronto

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


### Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues.

In [65]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Neighbourhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [66]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1591, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


### checking how many venues were returned for each neighbourhood

In [67]:
Toronto_venues.groupby('Neighborhood')['Venue'].count()


Neighborhood
Adelaide,King,Richmond                                                                                  100
Berczy Park                                                                                              58
Brockton,Exhibition Place,Parkdale Village                                                               24
Business Reply Mail Processing Centre 969 Eastern                                                        14
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara     15
Cabbagetown,St. James Town                                                                               42
Central Bay Street                                                                                       65
Chinatown,Grange Park,Kensington Market                                                                  64
Christie                                                                                                 17
Church and Well

In [68]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))


There are 235 uniques categories.


### Analyse Each Neighbourhood

In [69]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
Toronto_onehot.shape


(1591, 236)

### Let's group rows by neighbourhood and by taking the mean of the frequency of occurence of each category

In [71]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head(5)

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.02,0.0,...,0.01,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,...,0.00,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.041667
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.00,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000


### Let's print each neighbourhood along with top 5 most common venues¶

In [72]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.05
2           Hotel  0.04
3  Clothing Store  0.04
4             Gym  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.03
2        Cocktail Bar  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2       Nightclub  0.08
3  Breakfast Spot  0.08
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.14
1           Pizza Place  0.07
2            Restaurant  0.07
3  Fast Food Restaurant  0.07
4            Skate Park  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  freq
0      Airport Service  0.20
1       Airport Lounge  0.13


### Let's put the result above in the dataframe

In [73]:
#sorting the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Let's create a new dataframe and display the top 10 venues for each neighbourhood

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Hotel,Gym,Clothing Store,Bar,Thai Restaurant,Steakhouse,Lounge
1,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Bakery,Farmers Market,Restaurant,Café,Beer Bar,Cocktail Bar,Diner
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Park,Gym / Fitness Center,Brewery,Comic Shop,Pizza Place,Farmers Market,Fast Food Restaurant,Restaurant,Garden Center
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Harbor / Marina,Plane,Airport,Airport Food Court,Airport Gate,Airport Terminal,Sculpture Garden,Boutique


### Cluster Neighbourhoods

In [75]:
# Run K-means to cluster the neighbourhood into 5 clusters

kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [77]:
# add clustering labels
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Hotel,Gym,Clothing Store,Bar,Thai Restaurant,Steakhouse,Lounge
1,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Bakery,Farmers Market,Restaurant,Café,Beer Bar,Cocktail Bar,Diner
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Park,Gym / Fitness Center,Brewery,Comic Shop,Pizza Place,Farmers Market,Fast Food Restaurant,Restaurant,Garden Center
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Harbor / Marina,Plane,Airport,Airport Food Court,Airport Gate,Airport Terminal,Sculpture Garden,Boutique
5,"Cabbagetown,St. James Town",Pizza Place,Coffee Shop,Restaurant,Pub,Bakery,Italian Restaurant,Market,Café,Japanese Restaurant,Jewelry Store
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Department Store,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio
7,"Chinatown,Grange Park,Kensington Market",Coffee Shop,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Gaming Cafe,Park,Bakery,Dessert Shop,Caribbean Restaurant
8,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Nightclub
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


In [78]:
neighbourhoods_venues_sorted = neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:
df_Toronto.head(2)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188


In [80]:
Toronto_merged = df_Toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighbourhood')
Toronto_merged.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Nightclub,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,0,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Bookstore,Flea Market,Speakeasy,Italian Restaurant
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,0,Breakfast Spot,Gift Shop,Bookstore,Italian Restaurant,Restaurant,Dog Run,Dessert Shop,Movie Theater,Eastern European Restaurant,Cuban Restaurant
36,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,0,Café,Coffee Shop,Pizza Place,Pub,Italian Restaurant,Sushi Restaurant,Smoothie Shop,Bar,Indie Movie Theater,Spa
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Light Rail Station,Park,Gym / Fitness Center,Brewery,Comic Shop,Pizza Place,Farmers Market,Fast Food Restaurant,Restaurant,Garden Center


### Let's visualise the resulting clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters.save("map_clusters.html")
map_clusters

## Examine Clusters

In [82]:
cluster_0 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

cluster_0

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,East Toronto,0,Park,Pub,Liquor Store,Sandwich Place,Burrito Place,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Restaurant,Steakhouse
3,East Toronto,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,Central Toronto,0,Park,Lawyer,Swim School,Bus Line,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
5,Central Toronto,0,Gym,Park,Department Store,Pizza Place,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Doner Restaurant,Dog Run
6,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Health & Beauty Service,Ice Cream Shop,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Park
7,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Gym,Café,Italian Restaurant,Park,Pharmacy
9,Central Toronto,0,Pub,Coffee Shop,Bank,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Restaurant,Pizza Place
11,Downtown Toronto,0,Pizza Place,Coffee Shop,Restaurant,Pub,Bakery,Italian Restaurant,Market,Café,Japanese Restaurant,Jewelry Store
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


### observation cluster_0

Cluster_0 represents the busiest areas because of the presence of airport and aquariums. Coffeshops is also the 1st most common venue here indicating a good business because of the travelling customers.

### Cluster_1

In [83]:
cluster_1 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

cluster_1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Tennis Court,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### observation from cluster_1:
Cluster_1 seems to be more of a open land with parks, trail, trail as the most common top 3 common venue. It seems it is easily accessbile via the bus lines.Also, the presence of restaurants and shops gives a good indication of business around and rough ideas on the estimated visitors for this area.

### Cluster_2

In [84]:
cluster_2 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [85]:
cluster_2


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### observation cluster_2:
Gym is the most common venues in cluster_2 indicating people living in this cluster are much more into fitness and health.

### Cluster_3

In [86]:
cluster_3 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

cluster_3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Trail,Health Food Store,Neighborhood,Pub,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio


### observation from cluster_3
Grocery store is the most common venues in cluster_3 giving us an idea this cluster might include residential areas.There are cafes, parks, restaurants, athletics & sports and nightclubs. It has all the features of being a town centre too.

### Cluster _4

In [87]:
cluster_4 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

cluster_4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### observation cluster_4
Cluster_4 has the 1st Most Common venue as Pool. So, this area might be far away from the beach giving business opportunity for pools.